In [1]:
from collections import Counter
import re

file = ["www.google.com",
        "www.facebook.com",
        "www.youtube.com",
        "www.baidu.com",
        "www.yahoo.com",
        "www.amazon.com",
        "www.wikipedia.org",
        "www.qq.com",
        "www.taobao.com",
        "www.twitter.com",
        "www.google.co.in",
        "www.live.com",
        "www.sina.com.cn",
        "www.linkedin.com",
        "www.weibo.com",
        "www.yahoo.co.jp",
        "www.google.co.jp",
        "www.google.ca",
        "www.ok.ru",
        "www.adcash.com",
        "www.google.com.mx",
        "www.diply.com",
        "www.tianya.cn",
        "www.google.com.hk",
        "www.pornhub.com",
        "www.alibaba.com",
        "www.rakuten.co.jp",
        "www.naver.com",
        "www.amazon.co.uk",
        "www.google.com.tr",
        "www.adobe.com",
        "www.xinhuanet.com",
        "www.gmail.com",
        "www.outbrain.com",
        "www.xhamster.com",
        "www.detail.tmall.com",
        "www.soso.com",
        "www.google.co.id",
        "www.ebay.de",
        "www.kat.cr",
        "www.people.com.cn",
        "www.google.pl",
        "www.jd.com",
        "www.cntv.cn",
        "www.gmw.cn",
        "www.google.com.au",
        "www.go.com",
        "www.nicovideo.jp",
        "www.flipkart.com",
        "www.cnn.com",
        "www.dailymotion.com",
        "www.bbc.co.uk",
        "www.booking.com",
        "www.github.com",
        "www.googleusercontent.com",
        "www.pixnet.net",
        "www.dropbox.com",
        "www.wikia.com",
        "www.163.com",
        "www.chinadaily.com.cn"
        ]

category = {}
category["www.google.com"] = 0
category["www.facebook.com"] = 1
category["www.youtube.com"] = 2
category["www.baidu.com"] = 0
category["www.yahoo.com"] = 1
category["www.amazon.com"] = 2
category["www.wikipedia.org"] = 3
category["www.twitter.com"] = 1
category["www.google.co.in"] = 0
category["www.linkedin.com"] = 1
category["www.yahoo.co.jp"] = 1
category["www.google.co.jp"] = 0
category["www.google.ca"] = 0
category["www.qq.com"] = 2
category["www.taobao.com"] = 2
category["www.live.com"] = 1
category["www.google.com.mx"] = 0
category["www.google.com.hk"] = 0
category["www.pornhub.com"] = 2
category["www.alibaba.com"] = 2
category["www.amazon.co.uk"] = 2
category["www.google.com.tr"] = 0
category["www.adobe.com"] = 0
category["www.gmail.com"] = 0
category["www.xhamster.com"] = 2
category["www.ebay.de"] = 2
category["www.google.pl"] = 0
category["www.google.com.au"] = 0
category["www.dailymotion.com"] = 2
category["www.bbc.co.uk"] = 2
category["www.google.co.id"] = 0
category["www.flipkart.com"] = 2
category["www.cnn.com"] = 2
category["www.booking.com"] = 2
category["www.github.com"] = 0
category["www.dropbox.com"] = 0
category["www.people.com.cn"] = 2
category["www.nicovideo.jp"] = 2
category["www.googleusercontent.com"] = 0
category["www.weibo.com"] = 2
category["www.sina.com.cn"] = 2
category["www.adcash.com"] = 0
category["www.ok.ru"] = 2
category["www.diply.com"] = 2
category["www.tianya.cn"] = 2
category["www.jd.com"] = 2
category["www.cntv.cn"] = 0
category["www.go.com"] = 2
category["www.naver.com"] = 0
category["www.kat.cr"] = 0
category["www.soso.com"] = 0
category["www.wikia.com"] = 0
category["www.163.com"] = 2
category["www.gmw.cn"] = 2
category["www.pixnet.net"] = 3
category["www.outbrain.com"] = 3
category["www.xinhuanet.com"] = 2
category["www.rakuten.co.jp"] = 2
category["www.detail.tmall.com"] = 0
category["www.chinadaily.com.cn"] = 0

def find_PLT(filename):
    file = open(filename)
    list = file.readlines()
    mintime = 0
    for line in list:
        match = re.match('.*requestTime.: ([0-9^.]+)', line)
        if match is not None:
            mintime = float(match.groups(0)[0])
            break
    maxtime = 0
    for line in list:
        match = re.match('.*time.: ([0-9^.]+)',line)
        if match is not None:
            if float(match.groups(0)[0]) > maxtime:
                maxtime = float(match.groups(0)[0])
                
    length = 0
    for line in list:
        match = re.match('.*"len": ([0-9]+), "from',line)
        if match is not None:
            for val in match.groups(0):
                length += int(val)                
    
    return (maxtime - mintime,mintime,maxtime,length)
            
def findcount(text, list):
    feature = Counter()
    for i in range(4):
        feature[list[i]] = len(re.findall('\.' + list[i],text))
#         print list[i],":",feature[list[i]]
    for i in range(4,len(list)):
        feature[list[i]] = len(re.findall(list[i],text))
    return feature


def findAllPLT(name,count):
    list = []
    featureList = ['png','gif','css','js','Resource']
    feature = Counter()
    length = 0
    for i in range(count):
        filename = name + str(i)  + ".txt"
        text = open(filename).read()
        feature += findcount(text,featureList)
        (time,min,max,length) = find_PLT(filename)
        list.append(time)
    
    for key in featureList:
        if key in feature:
            feature[key] = (feature[key]*1.0)/count
        else:
            feature[key] = 0
    feature['length'] = length
    return feature,list

In [2]:
filepath = "/Users/Aman/Desktop/Colin/data_all/results/"
def all_file():
    dictionary = {}
    for website in file:
        feature,list = findAllPLT(filepath + website,10)
        dictionary[website] = (feature,list)
    return dictionary

all_val_dict = all_file()

In [6]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
file_1 = ["www.google.com",
        "www.facebook.com",
        "www.youtube.com",
        "www.baidu.com",
        "www.yahoo.com",
        "www.amazon.com",
        "www.wikipedia.org",
        "www.taobao.com",
        "www.twitter.com",
        "www.google.co.in",
        "www.live.com",
        "www.linkedin.com",
        "www.weibo.com",
        "www.yahoo.co.jp",
        "www.google.co.jp",
        "www.google.ca",
        "www.ok.ru",
        "www.adcash.com",
        "www.google.com.mx",
        "www.diply.com",
        "www.tianya.cn",
        "www.google.com.hk",
        "www.pornhub.com",
        "www.alibaba.com",
        "www.naver.com",
        "www.amazon.co.uk",
        "www.google.com.tr",
        "www.adobe.com",
        "www.gmail.com",
        "www.outbrain.com",
        "www.xhamster.com",
        "www.detail.tmall.com",
        "www.soso.com",
        "www.google.co.id",
        "www.ebay.de",
        "www.kat.cr",
        "www.google.pl",
        "www.cntv.cn",
        "www.gmw.cn",
        "www.google.com.au",
        "www.go.com",
        "www.nicovideo.jp",
        "www.flipkart.com",
        "www.cnn.com",
        "www.dailymotion.com",
        "www.bbc.co.uk",
        "www.booking.com",
        "www.googleusercontent.com",
        "www.pixnet.net",
        "www.dropbox.com",
        "www.wikia.com",
        "www.chinadaily.com.cn"
        ]
def pageloaditeration():
    all_val_dict['www.tianya.cn'][1].sort()
    all_val_dict['www.tianya.cn'][1].reverse()
    all_val_dict['www.adobe.com'][1].reverse()
    all_val_dict['www.pixnet.net'][1].reverse()

    
    for website in file_1:
        if website in all_val_dict.keys():
            if max(all_val_dict[website][1]) < 8 and min(all_val_dict[website][1])  > 0:
#                 all_val_dict[website][1].sort()
#                 all_val_dict[website][1].reverse()
                plt.plot(all_val_dict[website][1],linewidth=2.0)

    plt.ylabel('Time in seconds ---->')
    plt.xlabel('Iterations ---->')
    plt.ylim(ymax=8)
    plt.xlim(xmax=7)
    plt.grid(True)
    plt.title("PAGELOAD TIME  vs  ITERATION")
    plt.show()

def scatterplot(file):
    for website in file:
       plt.scatter(sum(all_val_dict[website][0].values()),np.mean(all_val_dict[website][1]),alpha=0.5,color="blue")

    plt.ylabel('Time in seconds ---->')
    plt.xlabel('Number of objects ---->')
    plt.grid(True)
    plt.title("PAGELOAD TIME  vs  #OBJECTS")
    plt.show()

def pageloadvsobj(all_val_dict):
    for website in file:
        sum = 0.0
        i = 0
        for key in all_val_dict[website][0]:
            sum += regr.coef_[i]* all_val_dict[website][0][key]
            i += 1
            sum = max(sum,2)
            plt.scatter(sum,np.mean(all_val_dict[website][1]),alpha=0.5)
    
    plt.ylabel('Time in seconds ---->')
    plt.xlabel('Number of objects ---->')
    plt.grid(True)
    plt.xlim(xmin=-5)
    plt.ylim(ymin=0)
    plt.title("PAGELOAD TIME  vs  #OBJECTS")
    plt.show()

def sp():
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    x = []
    y = []
    z = []
    yaxis = []
    keys = ['png','gif','css','js']
    for website in file:
        sum = 0
        for v in keys:
            sum += all_val_dict[website][0][v]
        x.append(sum)
        y.append(all_val_dict[website][0]["length"])
        z.append(all_val_dict[website][0]["Resource"])
        yaxis.append(np.mean(all_val_dict[website][1]))
    
    x = np.array(ss.zscore(x))
    y = np.array(ss.zscore(y))
    z = np.array(ss.zscore(z))
   
    colors = iter(cm.rainbow(np.linspace(0, 1, 3)))

    ax1.scatter(x,yaxis , s=35,color=next(colors), marker="o",edgecolor = 'black', label='#Objects')
    ax1.scatter(y,yaxis , s=35,color=next(colors), marker="s",edgecolor = 'black', label='LengthWebsite')
    ax1.scatter(z,yaxis , s=35,color=next(colors), marker="p",edgecolor = 'black', label='#Resources Contacted')
    plt.legend(loc='upper left');
    plt.colors()
    plt.ylabel('Time in seconds ---->')
    plt.xlabel('Features(NORMALIZED TO FIT/COMPARE) ---->')
    plt.grid(True)
    plt.title("PAGELOAD TIME vs Features")
    plt.show()

# pageloaditeration()
# pageloadvsobj(all_val_dict)
# scatterplot(file)
sp()

In [7]:
import numpy as np
from sklearn import datasets, linear_model
from scipy import stats as ss

def regression(Y,X):
    X = np.array(X)
    Y = np.array(Y)
    Y = np.array(ss.zscore(Y)).T
#     X = np.array([x + 0.1001 for x in X])
    X = np.array([(ss.zscore(x)) for x in X.T]).T
    
    regr = linear_model.LinearRegression()
    regr.fit(X,Y)
    print('Coefficients: \n', regr.coef_)
    print("Residual sum of squares: %.2f"% np.mean((regr.predict(X) - Y) ** 2))
    
    return regr    

def getValueRegr(all_val_dict):
    Y = []
    X = []
    for website in file:
        if np.sum(all_val_dict[website][1]) == 0:
            continue
        Y.append(np.mean(all_val_dict[website][1][4]))
        x1 = all_val_dict[website][0].values()
        
        X.append(x1)
    
    return (Y,X)

Y,X = getValueRegr(all_val_dict)

In [8]:
regr = regression(Y,X)
test = all_val_dict['www.yahoo.com'][0].values()
print test
# test = test[1:-1]
test = np.array(ss.zscore(test)).T
print test
print regr.predict(test)


for website in file:
    test = all_val_dict[website][0].values()
    test = np.array(ss.zscore(test)).T
    print regr.predict(test)
    print all_val_dict[website][1][0]


('Coefficients: \n', array([ 0.48527886,  0.31057993,  0.04810704, -0.19452292,  0.36876282,
       -0.3931335 ]))
Residual sum of squares: 0.41
[49.1, 168.2, 9.8, 3301150, 4.9, 52.4]
[-0.44721992 -0.44712311 -0.44725186  2.23606798 -0.44725585 -0.44721724]
[-0.80149151]
[-0.80149036]
1.663862
[-0.80149943]
1.520992
[-0.80147121]
2.995691
[-0.8014616]
1.413221
[-0.80149151]
3.552201
[-0.80150308]
3.856087
[-0.80135412]
1.388284
[-0.80144747]
6.975617
[ 0.7356777]
6.202976
[-0.80148759]
4.557441
[-0.80149123]
1.346912
[-0.80145364]
0.718564000001
[ 1.33964103]
14.220387
[-0.80147758]
1.516023
[-0.80138823]
3.14975
[-0.80140302]
7.131401
[-0.80149014]
1.318131
[-0.80148917]
1.053682
[-0.80147691]
3.536468
[-0.80150299]
5.341312
[-0.80148446]
1.059173
[-0.80148313]
3.90244
[-0.80141478]
7.356618
[-0.80149093]
1.642962
[-0.80149051]
4.289524
[-0.80153054]
5.16899
[-0.80147545]
0
[-0.8014739]
3.267857
[-0.80148511]
5.032134
[-0.80148866]
0.920662
[-0.80146387]
8.830258
[-0.80143299]
11.9142

/Users/Aman/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/Aman/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/Aman/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').